# Reflection Pattern 




In [3]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel, Field
from typing  import Literal
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ItemHelpers,
    TResponseInputItem
)
load_dotenv()

from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph


In [4]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)


## Defining the agents

Here we define two agents: 
    - Genrator Agent
    - Reviewer 

In [ ]:
class GeneratedContent(BaseModel):
    content: str = Field(description="The generated content for the presentation outline.")
    commentarytoevaluator: str = Field(description="Commentary to the evaluator about the generated content.")

generator_agent = Agent(
    name="generator_agent",
    model=model_name,
    instructions=(
        "You are a presentation outline generator agent. "
        "You will be provided with a prompt with presentation topic and audience. "
        "Your task is to generate a detailed outline for the presentation. "
        "If the prompt is not clear, ask clarifying questions to get more information. "
        "Always handoff to a single agent at a time."
    ),
    output_type=GeneratedContent
)

In [6]:
class EvaluationFeedback(BaseModel):
    feedback: str = Field(
        description="Feedback on the generated outline, including strengths and weaknesses."
    )
    score: Literal["pass", "needs_improvement", "fail"] = Field(
        description="Evaluation score for the generated outline."
        "pass: The outline meets the requirements and is well-structured. "
        "needs_improvement: The outline is acceptable but requires some adjustments. "
        "fail: The outline does not meet the requirements or is poorly structured."
    )

evaluation_agent = Agent(
    name="evaluation_agent",
    model=model_name,
    instructions=(
        "You are an evaluation agent. "
        "You will be provided with a presentation outline. "
        "Your task is to evaluate the outline based on the prompt and provide feedback "
        "and a score indicating whether the outline meets the requirements."
        "The feedback should include strengths and weaknesses of the outline, "
        "If it's not good enough, you provide feedback on what needs to be improved."
        "You dont seek perfection, you seek a good enough outline that demostrates"
        " thoughtful effort and ofcourse improvement. "
        "Never give it a pass on the first try."

    ),
    output_type=EvaluationFeedback,
)

In [7]:
msg = input("Enter the prompt for the presentation outline and audience: ") or (
            "Generate a presentation outline for a technical audience on"
             "the topic of AI in healthcare.")

input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_output : str | None = None
latest_feedback : EvaluationFeedback | None = None

i = 0
turn = 0


with trace(workflow_name="reflection_workflow", group_id="mdsi") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    while True:

        generated_outline = await Runner.run(
            generator_agent,
            input_items,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
            )
        )

        # print(generated_outline.final_output)
        # input_items += generated_outline.to_input_list()
        input_items.append(generated_outline.to_input_list()[-1])
        print(generated_outline)
        # intput_items = generated_outline.to_input_list()
        # latest_output = generated_outline.final_output
        generated_result : GeneratedContent = generated_outline.final_output
        # generated_result = generated_outline.final_output_as(GeneratedContent)
        latest_output = generated_result.content
        print(f"\n++ Commentary: {generated_result.commentarytoevaluator}")

        evaluation_result = await Runner.run(
            evaluation_agent,
            input_items,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
            ),
            max_turns=3,
        )
        turn += 1
        latest_feedback: EvaluationFeedback = evaluation_result.final_output
        print(f">> Evaluation Feedback (turn: {turn}): ", latest_feedback.feedback)
        print(">> Evaluation Score: ", latest_feedback.score)

        if latest_feedback.score == "pass":
            print(":: Outline is acceptable. Ending the loop.")
            break

        if i >= 3:
            print("!! Maximum iterations reached. Ending the loop.")
            break
        i += 1

        print(">< Outline is not acceptable. Re-run with feedback.")
        input_items.append(
            {
                "role": "user",
                "content": f"Feedback: {latest_feedback.feedback}"
            }
        )





Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_97795772bf964f2198a33b4980742112
RunResult:
- Last agent: Agent(name="generator_agent", ...)
- Final output (GeneratedContent):
    {
      "content": "### Presentation Outline: AI in Healthcare\n\n#### I. Introduction \n   A. Definition of Artificial Intelligence (AI)  \n   B. Overview of AI applications in various industries  \n   C. Importance of AI in the healthcare sector  \n   D. Objectives of the presentation  \n\n#### II. Background and Current Trends  \n   A. Historical perspective on AI in healthcare  \n      1. Early examples of AI in medical diagnostics  \n      2. Evolution of algorithms and machine learning techniques  \n   B. Current statistics on AI adoption in healthcare  \n   C. Trends driving AI integration into healthcare systems  \n      1. Increased data generation from wearable devices  \n      2. The rise of telemedicine  \n      3. Need for personalized medicine  \n\n#### III. Key Applications

In [10]:
display(Markdown(latest_output))

### Enhanced Presentation Outline: AI in Healthcare

#### I. Introduction
   A. Definition of Artificial Intelligence (AI)  
   B. Significance of AI in modern healthcare  
   C. Objectives of the presentation  

#### II. Background and Current Trends
   A. Historical overview of AI in healthcare  
      1. From early rule-based systems to machine learning  
      2. Key milestones and breakthroughs  
   B. Current statistics on AI adoption in healthcare  
      1. Industry reports and adoption rates  
   C. Trends driving AI integration into healthcare systems  
      1. Increased availability of healthcare data  
      2. Advances in computational power and algorithmic complexity  

#### III. Key Applications of AI in Healthcare  
   A. Medical Imaging  
      1. Algorithms: 
         - Convolutional Neural Networks (CNNs) for image recognition 
         - Generative Adversarial Networks (GANs) for data augmentation  
      2. Case studies: 
         - AI in detecting lung cancer and diabetic retinopathy  
   B. Clinical Decision Support Systems (CDSS)  
      1. Technologies used: 
         - IBM Watson Health, Google DeepMind  
      2. Impact on clinical outcomes, error reduction, and patient safety  
   C. Predictive Analytics  
      1. Techniques: 
         - Machine Learning algorithms for patient monitoring  
      2. Case studies: 
         - AI models predicting hospital readmissions and disease outbreaks  
   D. Drug Discovery and Development  
      1. In silico modeling and molecular simulations 
      2. Examples: collaborations between AI companies and pharma (e.g., BenevolentAI)  

#### IV. Ethical Considerations and Challenges of AI in Healthcare  
   A. Data Privacy and Security  
      1. Compliance with regulations (e.g., HIPAA, GDPR) 
   B. Ethical implications of AI use  
      1. Algorithmic bias and fairness  
         - Example: race and gender disparities in AI training datasets  
      2. Transparency and accountability in decision-making 
   C. Integration issues  
      1. Barriers to acceptance among healthcare professionals  
      2. Interoperability challenges with existing systems  
   D. Reliability and accuracy  
      1. Importance of continuous validation and performance monitoring 

#### V. Future Directions in AI for Healthcare  
   A. Emerging Technologies and Innovations  
      1. Natural language processing (NLP) for unstructured data analysis  
      2. Robotics in surgical applications and rehabilitation  
   B. Policy recommendations and frameworks for ethical AI use  
   C. Workforce impact analysis  
      1. Concerns about job displacement vs. upskilling opportunities  
      2. New roles created by AI integration in healthcare  

#### VI. Conclusion  
   A. Summary of key points discussed  
   B. The potential of AI to transform healthcare delivery and improve patient outcomes  
   C. Call to action for stakeholders to consider ethical implications and invest in responsible AI practices  

#### VII. Q&A Session  
   A. Open floor for audience questions  
   B. Discussion of audience experiences and concerns regarding AI in healthcare  

#### VIII. References  
   A. List of relevant research, articles, and resources for further reading  
   B. Suggested materials for follow-up discussions and insights.

## Implement with as tools
inspect the use of max turns


In [12]:
reflection_orchestrator_agent = Agent(
    name="reflection_orchestrator_agent",
    model=model_name,
    instructions=(
        "You are a reflection orchestrator agent. "
        "You will be provided with a prompt with presentation topic and audience. "
        "Leverage the specialised skills in the available tools to complete this task"
    ),
    tools=[
        generator_agent.as_tool(
            tool_name="outline_generator",
            tool_description="Generates a detailed outline for the presentation.",
            custom_output_extractor=None

        ),
        evaluation_agent.as_tool(
            tool_name="outline_evaluator",
            tool_description="Evaluates the generated outline and provides feedback.",
            custom_output_extractor=None
        )
    ],
    tool_use_behavior="run_llm_again",

)

try:
    draw_graph(reflection_orchestrator_agent, filename="viz/10_reflection_as_tools.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [14]:
from agents import RunContextWrapper, RunHooks, MaxTurnsExceeded

class ReflectionContext:
    turns: int = 0
    max_turns: int = 3

class ReflectionHooks(RunHooks):
    def __init__(self, max_turns: int):
        super().__init__()
        self.turns = 0
        self.max_turns = max_turns # can be used a constant as it doesnot really maintain state

    def on_agent_end(self, context, agent, output):
        print(f"agent name : {agent.name}")
        print(f"turns : {self.turns}")
        self.turns += 1  # does not maintain state

        reflectionContext : ReflectionContext = context.context
        reflectionContext.turns += 1 # this is how you would manage context between turns
        print(f"context turns : {reflectionContext.turns}")

        if reflectionContext.turns >= reflectionContext.max_turns:
            print("!! Maximum iterations reached. Ending the loop.")
            raise MaxTurnsExceeded(f"This reflection agent has reached its turns limit of {reflectionContext.max_turns}")

        return super().on_agent_end(context, agent, output)

with trace(workflow_name="reflection_as_tools_workflow", group_id="mdsi") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    msg = input ("Enter your message: ") or "How to prepare a presentation outline? for post grad students"

    input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

    reflection_result = None
    reflection_context = ReflectionContext() # default max truns == 3
    while True:
        try:
            reflection_result = await Runner.run(
                reflection_orchestrator_agent,
                input_items,
                run_config=RunConfig(
                    model=model_name,
                    model_provider=GITHUB_MODEL_PROVIDER,

                ),
                context=reflection_context,
                hooks=ReflectionHooks(max_turns=3),
                max_turns=3
            )
        except MaxTurnsExceeded as e:
            print(f"!! Maximum iterations reached. Ending the loop. {e}")
            break

    print(reflection_result)

Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_2d47dc10ebeb49ab93a29aefa72a79ba
agent name : reflection_orchestrator_agent
turns : 0
context turns : 1
agent name : reflection_orchestrator_agent
turns : 0
context turns : 2
agent name : reflection_orchestrator_agent
turns : 0
context turns : 3
!! Maximum iterations reached. Ending the loop.
!! Maximum iterations reached. Ending the loop. This reflection agent has reached its turns limit of 3
RunResult:
- Last agent: Agent(name="reflection_orchestrator_agent", ...)
- Final output (str):
    ### Presentation Outline: Advanced Research Methodologies in Postgraduate Studies
    
    #### I. Introduction  
       A. Greeting and Introduction  
          1. Presenter name and background  
          2. Importance of research methodologies in postgraduate studies  
       B. Objective of the Presentation  
          1. To provide an overview of advanced research methodologies  
          2. To prepare students for conductin